In [1]:
pip install -U transformers datasets evaluate nltk accelerate trl wandb peft rouge_score

  Obtaining dependency information for wandb from https://files.pythonhosted.org/packages/45/e0/7ba3b78a74413b7467300cb7a5d486b9871ee464a7cade98ea869d3ca3df/wandb-0.19.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pydantic<3,>=2.6 from https://files.pythonhosted.org/packages/62/51/72c18c55cf2f46ff4f91ebcc8f75aa30f7305f3d726be3f4ebffb4ae972b/pydantic-2.10.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 14.5 MB/s eta 0:00:00
  Obtaining dependency information for annotated-types>=0.6.0 from https://files.pythonhosted.org/packages/78/b6/6307fbef88d9b5ee7421e68d78a9f162e0da4900bc5f5793f6d3d0e34fb8/annotated_types-0.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic-core==2.27.1 from https://files.pythonhosted.org/packages/85/d7/936846087424c882d89c853711687230cd60179a67c79c34c99b64f92625/pydantic_core-2.27.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.m

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
from peft import PeftModel, PeftConfig, LoraConfig, TaskType, get_peft_model

model_name = "google/flan-t5-base"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.truncate_side = "left"
model.config.max_length = 256

In [7]:
from datasets import load_dataset
import numpy as np

dataset = load_dataset("Jise/hh-rlhf-helpful-base")

def preprocess(examples):
    inputs = ["".join([m["role"].capitalize() + m["content"] + "\n\n" for m in x]) + "Assistant:" for x in examples["prompt"]]
    model_inputs = tokenizer(inputs, max_length=511, truncation=True)
    labels = [x[0]["content"] for x in examples["chosen"]]
    labels = tokenizer(text_target=labels, max_length=256, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

dataset = dataset.map(preprocess, batched=True)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'prompt', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 43835
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'prompt', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2354
    })
})


In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from trl import DPOTrainer, DPOConfig
from trl.trainer.utils import SIMPLE_CHAT_TEMPLATE
import requests
import pickle
from transformers.optimization import Adafactor, AdafactorSchedule
import wandb
import nltk
import evaluate

nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True)
metric = evaluate.load("rouge")

run_name = "Flan-T5_DPO_LoRA_HH-RLHF"

with open("TOKENS.pkl", "rb") as f:
    TOKENS = pickle.load(f)

WANDB_TOKEN = TOKENS["WANDB_TOKEN"]
HF_TOKEN = TOKENS["HF_TOKEN"]

wandb.login(key=WANDB_TOKEN)

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

sft_training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-sft-lora",
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    weight_decay=0.01,
    bf16=True,
    predict_with_generate=True,
    push_to_hub=True,
    report_to="wandb",
    run_name=run_name,
    hub_token=HF_TOKEN,
    hub_model_id="Jise/flan-t5-hh-dpo-lora",
    save_safetensors=False,
)

sft_trainer = Seq2SeqTrainer(
    model=model,
    args=sft_training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/idies/.netrc
/tmp/ipykernel_2876/1791617875.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  sft_trainer = Seq2SeqTrainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
sft_trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1000,2.457000,2.269583,0.199893,0.063281,0.163376,0.177872
2000,2.418700,2.253200,0.194891,0.062697,0.160896,0.174012
3000,2.328900,2.243706,0.208327,0.065260,0.169937,0.185981
4000,2.347300,2.236511,0.205002,0.065210,0.167471,0.183720
5000,2.309000,2.230092,0.204255,0.063690,0.166663,0.182348
6000,2.270100,2.229096,0.208357,0.066447,0.169990,0.185868
7000,2.287000,2.226198,0.206204,0.065627,0.168390,0.184685
8000,2.258700,2.224727,0.208840,0.066966,0.169965,0.186928


/home/idies/miniconda3/envs/py39/lib/python3.9/site-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tra

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
from peft import PeftModel, PeftConfig, LoraConfig, TaskType, get_peft_model
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from trl import DPOTrainer, DPOConfig
from trl.trainer.utils import SIMPLE_CHAT_TEMPLATE
import requests
import pickle
from transformers.optimization import Adafactor, AdafactorSchedule
import wandb
import nltk
import evaluate
from datasets import load_dataset
import numpy as np

checkpoint = "./flan-t5-sft-lora/checkpoint-8220"

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

dataset = load_dataset("Jise/hh-rlhf-helpful-base")

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

if tokenizer.chat_template is None:
    tokenizer.chat_template = SIMPLE_CHAT_TEMPLATE

nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True)
metric = evaluate.load("rouge")

run_name = "Flan-T5_DPO_LoRA_HH-RLHF"

with open("TOKENS.pkl", "rb") as f:
    TOKENS = pickle.load(f)

WANDB_TOKEN = TOKENS["WANDB_TOKEN"]
HF_TOKEN = TOKENS["HF_TOKEN"]

wandb.login(key=WANDB_TOKEN)

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

peft_model = get_peft_model(model, lora_config)

dpo_training_args = DPOConfig(
    output_dir="./flan-t5-dpo-lora",
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=6,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=1e-5,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=100,
    weight_decay=0.01,
    max_prompt_length=511,
    max_length=256,
    save_total_limit=1,
    truncation_mode='keep_end',
    bf16=True,
    push_to_hub=True,
    report_to="wandb",
    run_name=run_name,
    hub_token=HF_TOKEN,
    hub_model_id="Jise/flan-t5-hh-dpo-lora",
    save_safetensors=False,
)


dpo_trainer = DPOTrainer(
    model=peft_model,
    args=dpo_training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
)

2024-12-05 04:06:07.555081: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-05 04:06:08.142034: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/idies/miniconda3/envs/py39/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jiseshen 

In [3]:
dpo_trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/idies/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
100,0.707400,0.692520,-0.001593,-0.002884,0.540713,0.001291,-148.483109,-118.399284,-17.018328,-17.126493
200,0.710800,0.690911,-0.005331,-0.010041,0.593723,0.004710,-148.520462,-118.470856,-17.053373,-17.161543
300,0.701400,0.688665,-0.012032,-0.021987,0.600085,0.009955,-148.587494,-118.590332,-17.110165,-17.218410
400,0.697900,0.686078,-0.022928,-0.039478,0.609839,0.016550,-148.696442,-118.765236,-17.188845,-17.298573
500,0.688500,0.683100,-0.039983,-0.064101,0.612383,0.024118,-148.866989,-119.011452,-17.290449,-17.401979
600,0.680000,0.680043,-0.059577,-0.092961,0.606870,0.033384,-149.062943,-119.300056,-17.406548,-17.521002
700,0.681700,0.677302,-0.082866,-0.125386,0.613232,0.042520,-149.295837,-119.624313,-17.529215,-17.647436
800,0.666700,0.674662,-0.107003,-0.158802,0.614928,0.051798,-149.537201,-119.958473,-17.641436,-17.763874
900,0.654700,0.673123,-0.138239,-0.197792,0.610687,0.059553,-149.849548,-120.348358,-17.771891,-17.898695
1000,0.650200,0.670639,-0.173015,-0.242611,0.614080,0.069596,-150.197327,-120.796570,-17.897585,-18.029146


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/home/idies/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/home/idies/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/home/idies/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/home/idies/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/home/idies

TrainOutput(global_step=8217, training_loss=0.5805910762040825, metrics={'train_runtime': 43558.6142, 'train_samples_per_second': 3.019, 'train_steps_per_second': 0.189, 'total_flos': 0.0, 'train_loss': 0.5805910762040825, 'epoch': 2.9991787571858746})

In [4]:
results = dpo_trainer.evaluate()
print(results)

trainer.save_model("./flan-t5-dpo-trained")
tokenizer.save_pretrained("./flan-t5-dpo-trained")

{'eval_loss': 0.6549020409584045, 'eval_runtime': 233.4999, 'eval_samples_per_second': 10.081, 'eval_steps_per_second': 1.683, 'eval_rewards/chosen': -1.223189353942871, 'eval_rewards/rejected': -1.6155133247375488, 'eval_rewards/accuracies': 0.6458864212036133, 'eval_rewards/margins': 0.39232388138771057, 'eval_logps/chosen': -160.6990509033203, 'eval_logps/rejected': -134.52557373046875, 'eval_logits/chosen': -20.92389488220215, 'eval_logits/rejected': -21.557830810546875, 'epoch': 2.9991787571858746}


NameError: name 'trainer' is not defined